In [1]:
import torch
import time
import numpy as np
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
torch_rand = torch.rand(100, 100,100,100).to(device)
tord_rand_2 = torch.rand(100, 100,100,100).to(device)
np_rand = np.random.rand(100, 100,100,100)
np_rand_2 = np.random.rand(100, 100,100,100)

start_time = time.time()

rand = torch_rand @ tord_rand_2
end_time = time.time()
elapsed_time = end_time - start_time
print(f"torch time: {elapsed_time} seconds")

start_time = time.time()
rand=np.multiply(np_rand, np_rand_2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"numpy time: {elapsed_time} seconds")











torch time: 0.030534982681274414 seconds
numpy time: 0.3213059902191162 seconds


In [8]:
%%time
start_time= time.time()
zeros=torch.zeros(1,1)
end_time =  time.time()
elapsed_time = end_time - start_time
print(f"torch time: {elapsed_time} seconds")    







torch time: 0.0 seconds
CPU times: total: 0 ns
Wall time: 1 ms
